Notebook which gets known features for GW 1 of 2019/20 season - needed to predict points based on information known at GW 38 of previous season

In [1]:
import pandas as pd
import numpy as np
import os
import unidecode

pd.set_option('display.max_columns', None)

In [2]:
os.chdir('/Users/stevengeorge/Documents/Github/fpl-analysis')
DATA_DIR = 'data/external/FantasyPremierLeague/2019-20/'

In [59]:
players_raw = pd.read_csv(DATA_DIR + 'players_raw.csv',)

In [60]:
players_raw.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,id,in_dreamteam,influence,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,0,5,475,None,None,6,69140,0,0,0,0,106.0,0,2,None,None,0,Shkodran,0.0,40,2,107.8,1,False,718.6,2611,NaN,None,55,0,0,0,69140.jpg,2.6,0,0,Mustafi,0.4,False,None,a,1,3,252.0,80,0,0,0,0,0.0,0.0,Mustafi,9
1,5,5,304,0,None,4,98745,0,0,0,0,197.8,0,2,None,None,0,Héctor,0.0,21,0,73.7,2,False,261.6,1532,Knee injury - Unknown return date,2019-06-25T11:30:06.897107Z,55,1,0,0,98745.jpg,3.2,0,0,Bellerín,0.3,False,None,i,1,3,280.0,60,0,0,0,0,0.0,0.0,Bellerín,3
2,7,10,382,None,None,5,111457,0,0,0,0,445.9,0,2,None,None,0,Sead,0.0,29,0,101.0,3,False,323.2,1886,NaN,None,55,1,0,0,111457.jpg,3.4,0,0,Kolasinac,1.6,False,None,a,1,3,242.0,81,0,0,0,0,0.0,0.0,Kolasinac,5
3,1,3,152,None,None,2,154043,0,0,0,0,98.9,0,2,None,None,0,Ainsley,0.0,20,1,44.8,4,False,224.2,983,NaN,None,50,0,0,0,154043.jpg,2.1,1,0,Maitland-Niles,0.8,False,None,a,1,3,126.0,34,0,0,0,0,0.0,0.0,Maitland-Niles,1
4,2,4,402,None,None,5,39476,0,0,0,0,67.9,0,2,None,None,0,Sokratis,0.0,33,1,70.4,5,False,513.8,2198,NaN,None,50,0,0,0,39476.jpg,2.6,0,0,Papastathopoulos,1.1,False,None,a,1,3,120.0,64,0,0,0,0,0.0,0.0,Sokratis,12


In [61]:
players_raw = players_raw[['first_name', 'now_cost', 'second_name', 'team', 'element_type']]

POSITION_MAP = { # https://github.com/vaastav/Fantasy-Premier-League/issues/1
    1: 'GK',
    2: 'DEF',
    3: 'MID',
    4: 'FWD'
}
players_raw['element_type'] = players_raw['element_type'].map(POSITION_MAP)

players_raw.rename(columns={'element_type': 'position'}, inplace=True)

players_raw.head()

,first_name,now_cost,second_name,team,position
0,Shkodran,55,Mustafi,1,DEF
1,Héctor,55,Bellerín,1,DEF
2,Sead,55,Kolasinac,1,DEF
3,Ainsley,50,Maitland-Niles,1,DEF
4,Sokratis,50,Papastathopoulos,1,DEF


In [62]:
fixtures = pd.read_csv(DATA_DIR + 'fixtures.csv')
fixtures.head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,stats,team_a,team_a_difficulty,team_a_score,team_h,team_h_difficulty,team_h_score
0,1059702,1,False,False,1,2019-08-09T19:00:00Z,0,False,False,[],14,5,NaN,10,2,NaN
1,1059709,1,False,False,8,2019-08-10T11:30:00Z,0,False,False,[],11,3,NaN,19,4,NaN
2,1059703,1,False,False,2,2019-08-10T14:00:00Z,0,False,False,[],15,3,NaN,3,2,NaN
3,1059704,1,False,False,3,2019-08-10T14:00:00Z,0,False,False,[],16,2,NaN,5,2,NaN
4,1059705,1,False,False,4,2019-08-10T14:00:00Z,0,False,False,[],8,2,NaN,7,2,NaN


In [63]:
print(fixtures.shape)
fixtures['event'].unique()  # event is gw

(380, 16)


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38])

In [64]:
fixtures = fixtures[['event', 'kickoff_time', 'team_a', 'team_h']]
fixtures.rename(columns={'event': 'gw'}, inplace=True)

fixtures = fixtures[fixtures['gw'] == 1]  # keep only GW 1

fixtures['kickoff_time'] = fixtures['kickoff_time'].str.replace('T', ' ')
fixtures['kickoff_time'] = pd.to_datetime(fixtures['kickoff_time'])
fixtures['kickoff_time'] = fixtures['kickoff_time'].dt.tz_convert('Europe/London')

In [65]:
print(fixtures.shape)
fixtures

(10, 4)


,gw,kickoff_time,team_a,team_h
0,1,2019-08-09 20:00:00+01:00,14,10
1,1,2019-08-10 12:30:00+01:00,11,19
2,1,2019-08-10 15:00:00+01:00,15,3
3,1,2019-08-10 15:00:00+01:00,16,5
4,1,2019-08-10 15:00:00+01:00,8,7
5,1,2019-08-10 15:00:00+01:00,4,18
6,1,2019-08-10 17:30:00+01:00,2,17
7,1,2019-08-11 14:00:00+01:00,20,9
8,1,2019-08-11 14:00:00+01:00,1,13
9,1,2019-08-11 16:30:00+01:00,6,12


In [91]:
away_fixtures = fixtures[['gw', 'kickoff_time', 'team_a']]
away_fixtures['was_home'] = False
away_fixtures.rename(columns={'team_a': 'team'}, inplace=True)

home_fixtures = fixtures[['gw', 'kickoff_time', 'team_h']]
home_fixtures['was_home'] = True
home_fixtures.rename(columns={'team_h': 'team'}, inplace=True)

each_team_fixtures = away_fixtures.append(home_fixtures, ignore_index=True)
each_team_fixtures

,gw,kickoff_time,team,was_home
0,1,2019-08-09 20:00:00+01:00,14,False
1,1,2019-08-10 12:30:00+01:00,11,False
2,1,2019-08-10 15:00:00+01:00,15,False
3,1,2019-08-10 15:00:00+01:00,16,False
4,1,2019-08-10 15:00:00+01:00,8,False
5,1,2019-08-10 15:00:00+01:00,4,False
6,1,2019-08-10 17:30:00+01:00,2,False
7,1,2019-08-11 14:00:00+01:00,20,False
8,1,2019-08-11 14:00:00+01:00,1,False
9,1,2019-08-11 16:30:00+01:00,6,False


In [67]:
print(players_raw.shape)
players_raw = players_raw.merge(each_team_fixtures, on='team', how='left')
print(players_raw.shape)

players_raw.head()

(454, 5)
(454, 8)


,first_name,now_cost,second_name,team,position,gw,kickoff_time,was_home
0,Shkodran,55,Mustafi,1,DEF,1,2019-08-11 14:00:00+01:00,False
1,Héctor,55,Bellerín,1,DEF,1,2019-08-11 14:00:00+01:00,False
2,Sead,55,Kolasinac,1,DEF,1,2019-08-11 14:00:00+01:00,False
3,Ainsley,50,Maitland-Niles,1,DEF,1,2019-08-11 14:00:00+01:00,False
4,Sokratis,50,Papastathopoulos,1,DEF,1,2019-08-11 14:00:00+01:00,False


In [68]:
players_raw['name'] = players_raw['first_name'] + '_' + players_raw['second_name']

players_raw['name'] = players_raw['name'].apply(lambda x: unidecode.unidecode(x))
players_raw['name'] = players_raw['name'].str.lower()

In [69]:
players_raw.head()

,first_name,now_cost,second_name,team,position,gw,kickoff_time,was_home,name
0,Shkodran,55,Mustafi,1,DEF,1,2019-08-11 14:00:00+01:00,False,shkodran_mustafi
1,Héctor,55,Bellerín,1,DEF,1,2019-08-11 14:00:00+01:00,False,hector_bellerin
2,Sead,55,Kolasinac,1,DEF,1,2019-08-11 14:00:00+01:00,False,sead_kolasinac
3,Ainsley,50,Maitland-Niles,1,DEF,1,2019-08-11 14:00:00+01:00,False,ainsley_maitland-niles
4,Sokratis,50,Papastathopoulos,1,DEF,1,2019-08-11 14:00:00+01:00,False,sokratis_papastathopoulos


__Create kickoff features__

In [70]:
players_raw['kickoff_month'] = players_raw['kickoff_time'].dt.strftime("%b")
players_raw = pd.get_dummies(players_raw, columns=['kickoff_month'])

for month in ['kickoff_month_Apr', 'kickoff_month_Dec', 'kickoff_month_Feb', 'kickoff_month_Jan',
              'kickoff_month_Mar', 'kickoff_month_May', 'kickoff_month_Nov', 'kickoff_month_Oct', 'kickoff_month_Sep']:
    players_raw[month] = 0

In [71]:
players_raw['kickoff_hour'] = players_raw['kickoff_time'].dt.hour

players_raw['late_kickoff'] = np.where(
    players_raw['kickoff_hour'] >= 17,
    1,
    0
)

players_raw['early_kickoff'] = np.where(
    players_raw['kickoff_hour'] <= 13,
    1,
    0
)

players_raw.drop(columns=['kickoff_hour'], axis=1, inplace=True)

players_raw.head()

,first_name,now_cost,second_name,team,position,gw,kickoff_time,was_home,name,kickoff_month_Aug,kickoff_month_Apr,kickoff_month_Dec,kickoff_month_Feb,kickoff_month_Jan,kickoff_month_Mar,kickoff_month_May,kickoff_month_Nov,kickoff_month_Oct,kickoff_month_Sep,late_kickoff,early_kickoff
0,Shkodran,55,Mustafi,1,DEF,1,2019-08-11 14:00:00+01:00,False,shkodran_mustafi,1,0,0,0,0,0,0,0,0,0,0,0
1,Héctor,55,Bellerín,1,DEF,1,2019-08-11 14:00:00+01:00,False,hector_bellerin,1,0,0,0,0,0,0,0,0,0,0,0
2,Sead,55,Kolasinac,1,DEF,1,2019-08-11 14:00:00+01:00,False,sead_kolasinac,1,0,0,0,0,0,0,0,0,0,0,0
3,Ainsley,50,Maitland-Niles,1,DEF,1,2019-08-11 14:00:00+01:00,False,ainsley_maitland-niles,1,0,0,0,0,0,0,0,0,0,0,0
4,Sokratis,50,Papastathopoulos,1,DEF,1,2019-08-11 14:00:00+01:00,False,sokratis_papastathopoulos,1,0,0,0,0,0,0,0,0,0,0,0


In [72]:
players_raw.rename(columns={'now_cost': 'value'}, inplace=True)
players_raw['value'] = players_raw['value'] / 10

In [73]:
players_raw['season'] = '2019-20'

In [74]:
players_raw = pd.get_dummies(players_raw, columns=['position'])
players_raw.head()

,first_name,value,second_name,team,gw,kickoff_time,was_home,name,kickoff_month_Aug,kickoff_month_Apr,kickoff_month_Dec,kickoff_month_Feb,kickoff_month_Jan,kickoff_month_Mar,kickoff_month_May,kickoff_month_Nov,kickoff_month_Oct,kickoff_month_Sep,late_kickoff,early_kickoff,season,position_DEF,position_FWD,position_GK,position_MID
0,Shkodran,5.5,Mustafi,1,1,2019-08-11 14:00:00+01:00,False,shkodran_mustafi,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0
1,Héctor,5.5,Bellerín,1,1,2019-08-11 14:00:00+01:00,False,hector_bellerin,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0
2,Sead,5.5,Kolasinac,1,1,2019-08-11 14:00:00+01:00,False,sead_kolasinac,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0
3,Ainsley,5.0,Maitland-Niles,1,1,2019-08-11 14:00:00+01:00,False,ainsley_maitland-niles,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0
4,Sokratis,5.0,Papastathopoulos,1,1,2019-08-11 14:00:00+01:00,False,sokratis_papastathopoulos,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0


__Get opponent data__

In [88]:
opponents = fixtures.copy()[['team_a', 'team_h']]
opponents.rename(columns={'team_a': 'team', 'team_h': 'opponent'}, inplace=True)
opponents

,team,opponent
0,14,10
1,11,19
2,15,3
3,16,5
4,8,7
5,4,18
6,2,17
7,20,9
8,1,13
9,6,12


In [89]:
reverse = fixtures.copy()[['team_h', 'team_a']]
reverse.rename(columns={'team_h': 'team', 'team_a': 'opponent'}, inplace=True)
reverse

,team,opponent
0,10,14
1,19,11
2,3,15
3,5,16
4,7,8
5,18,4
6,17,2
7,9,20
8,13,1
9,12,6


In [90]:
opponent_df = opponents.append(reverse, ignore_index=True)
opponent_df

,team,opponent
0,14,10
1,11,19
2,15,3
3,16,5
4,8,7
5,4,18
6,2,17
7,20,9
8,1,13
9,6,12


In [95]:
players_raw = players_raw.merge(opponent_df, on='team')
players_raw.head()

,first_name,value,second_name,team,gw,kickoff_time,was_home,name,kickoff_month_Aug,kickoff_month_Apr,kickoff_month_Dec,kickoff_month_Feb,kickoff_month_Jan,kickoff_month_Mar,kickoff_month_May,kickoff_month_Nov,kickoff_month_Oct,kickoff_month_Sep,late_kickoff,early_kickoff,season,position_DEF,position_FWD,position_GK,position_MID,opponent
0,Shkodran,5.5,Mustafi,1,1,2019-08-11 14:00:00+01:00,False,shkodran_mustafi,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13
1,Héctor,5.5,Bellerín,1,1,2019-08-11 14:00:00+01:00,False,hector_bellerin,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13
2,Sead,5.5,Kolasinac,1,1,2019-08-11 14:00:00+01:00,False,sead_kolasinac,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13
3,Ainsley,5.0,Maitland-Niles,1,1,2019-08-11 14:00:00+01:00,False,ainsley_maitland-niles,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13
4,Sokratis,5.0,Papastathopoulos,1,1,2019-08-11 14:00:00+01:00,False,sokratis_papastathopoulos,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13


__Get team and opponent team names and features__

In [81]:
team_data = pd.read_csv('data/external/team_season_data.csv')
team_data.head()

,team_name,team,season,promoted_side,top_6_last_season
0,Arsenal,1,2016-17,0,1
1,Bournemouth,2,2016-17,0,0
2,Burnley,3,2016-17,1,0
3,Chelsea,4,2016-17,0,0
4,Crystal Palace,5,2016-17,0,0


In [96]:
players_raw = players_raw.merge(team_data, on=['team', 'season'], how='left')
players_raw.head()

,first_name,value,second_name,team,gw,kickoff_time,was_home,name,kickoff_month_Aug,kickoff_month_Apr,kickoff_month_Dec,kickoff_month_Feb,kickoff_month_Jan,kickoff_month_Mar,kickoff_month_May,kickoff_month_Nov,kickoff_month_Oct,kickoff_month_Sep,late_kickoff,early_kickoff,season,position_DEF,position_FWD,position_GK,position_MID,opponent,team_name,promoted_side,top_6_last_season
0,Shkodran,5.5,Mustafi,1,1,2019-08-11 14:00:00+01:00,False,shkodran_mustafi,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13,Arsenal,0,1
1,Héctor,5.5,Bellerín,1,1,2019-08-11 14:00:00+01:00,False,hector_bellerin,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13,Arsenal,0,1
2,Sead,5.5,Kolasinac,1,1,2019-08-11 14:00:00+01:00,False,sead_kolasinac,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13,Arsenal,0,1
3,Ainsley,5.0,Maitland-Niles,1,1,2019-08-11 14:00:00+01:00,False,ainsley_maitland-niles,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13,Arsenal,0,1
4,Sokratis,5.0,Papastathopoulos,1,1,2019-08-11 14:00:00+01:00,False,sokratis_papastathopoulos,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13,Arsenal,0,1


In [97]:
players_raw = players_raw.merge(
    team_data,
    left_on=['opponent', 'season'],
    right_on=['team', 'season'],
    suffixes=('', '_opponent'),
    how='left'
)

Check that fixtures are correct for GW 1:

In [101]:
players_raw.groupby(['team_name', 'team_name_opponent']).count().reset_index()[['team_name', 'team_name_opponent']]

,team_name,team_name_opponent
0,Arsenal,Newcastle United
1,Aston Villa,Tottenham Hotspur
2,Bournemouth,Sheffield Utd
3,Brighton & Hove Albion,Watford
4,Burnley,Southampton
5,Chelsea,Manchester United
6,Crystal Palace,Everton
7,Everton,Crystal Palace
8,Leicester City,Wolverhampton Wanderers
9,Liverpool,Norwich


In [102]:
players_raw.head()

,first_name,value,second_name,team,gw,kickoff_time,was_home,name,kickoff_month_Aug,kickoff_month_Apr,kickoff_month_Dec,kickoff_month_Feb,kickoff_month_Jan,kickoff_month_Mar,kickoff_month_May,kickoff_month_Nov,kickoff_month_Oct,kickoff_month_Sep,late_kickoff,early_kickoff,season,position_DEF,position_FWD,position_GK,position_MID,opponent,team_name,promoted_side,top_6_last_season,team_name_opponent,team_opponent,promoted_side_opponent,top_6_last_season_opponent
0,Shkodran,5.5,Mustafi,1,1,2019-08-11 14:00:00+01:00,False,shkodran_mustafi,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13,Arsenal,0,1,Newcastle United,13,0,0
1,Héctor,5.5,Bellerín,1,1,2019-08-11 14:00:00+01:00,False,hector_bellerin,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13,Arsenal,0,1,Newcastle United,13,0,0
2,Sead,5.5,Kolasinac,1,1,2019-08-11 14:00:00+01:00,False,sead_kolasinac,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13,Arsenal,0,1,Newcastle United,13,0,0
3,Ainsley,5.0,Maitland-Niles,1,1,2019-08-11 14:00:00+01:00,False,ainsley_maitland-niles,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13,Arsenal,0,1,Newcastle United,13,0,0
4,Sokratis,5.0,Papastathopoulos,1,1,2019-08-11 14:00:00+01:00,False,sokratis_papastathopoulos,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,13,Arsenal,0,1,Newcastle United,13,0,0


__Drop irrelevant features to allign with other season data__

In [109]:
COLUMNS_TO_DROP = ['kickoff_time', 'team_opponent', 'first_name', 'second_name', 'team', 'opponent']

players_raw.drop(
    columns=COLUMNS_TO_DROP,
    axis=1,
    inplace=True
)
players_raw.head()

,value,gw,was_home,name,kickoff_month_Aug,kickoff_month_Apr,kickoff_month_Dec,kickoff_month_Feb,kickoff_month_Jan,kickoff_month_Mar,kickoff_month_May,kickoff_month_Nov,kickoff_month_Oct,kickoff_month_Sep,late_kickoff,early_kickoff,season,position_DEF,position_FWD,position_GK,position_MID,team_name,promoted_side,top_6_last_season,team_name_opponent,promoted_side_opponent,top_6_last_season_opponent
0,5.5,1,False,shkodran_mustafi,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,Arsenal,0,1,Newcastle United,0,0
1,5.5,1,False,hector_bellerin,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,Arsenal,0,1,Newcastle United,0,0
2,5.5,1,False,sead_kolasinac,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,Arsenal,0,1,Newcastle United,0,0
3,5.0,1,False,ainsley_maitland-niles,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,Arsenal,0,1,Newcastle United,0,0
4,5.0,1,False,sokratis_papastathopoulos,1,0,0,0,0,0,0,0,0,0,0,0,2019-20,1,0,0,0,Arsenal,0,1,Newcastle United,0,0


In [111]:
print(players_raw.shape)
players_raw.to_parquet('data/processed/gameweek_1_static_features_201920', index=False)

(454, 27)
